In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop('id',axis=1,inplace=True)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
## independend and dependen features
x =df.drop('price',axis=1)
y= df[['price']]

In [5]:
x.shape

(193573, 9)

In [6]:
y.shape

(193573, 1)

In [7]:
# sargating num and catagorical cols 
num_cols= x.select_dtypes(exclude='object').columns
category_cols= x.select_dtypes(include='object').columns


In [8]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
# Feature Engginearing automation
num_pipline=Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='mean')), # missing value handaling
        ('scaler',StandardScaler())
    ]
)
cat_pipeline=Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='most_frequent')),
        ('encoding',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
process=ColumnTransformer(
    [
        ('num_pipline',num_pipline,num_cols),
        ('cat_pipeline',cat_pipeline,category_cols)
    ]
)
process

ColumnTransformer(transformers=[('num_pipline',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [11]:
## train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.27,random_state=35)

In [12]:
x_train=pd.DataFrame(process.fit_transform(x_train),columns=process.get_feature_names_out())

x_test=pd.DataFrame(process.transform(x_test),columns=process.get_feature_names_out())

In [13]:
x_train.head(2)

,num_pipline__carat,num_pipline__depth,num_pipline__table,num_pipline__x,num_pipline__y,num_pipline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.559105,-0.945818,1.444305,1.539867,1.509023,1.402409,-0.132044,-0.316835,0.015886
1,0.974581,0.073573,-0.640705,1.025548,1.036832,1.025274,0.873988,1.531745,0.015886


In [14]:
## importing models
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
reg=LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [16]:
reg.coef_

array([[ 6415.56066649,  -129.40257081,   -66.17614007, -2027.99290193,
         -139.71012559,  -102.82270943,    72.41372273,  -464.05890235,
          652.79443395]])

In [17]:
reg.intercept_

array([3961.56505647])

In [18]:
# import numpy as np
# def evaluate_models(actual,predicted):
#     mse=mean_squared_error(actual,predicted)
#     mae=mean_absolute_error(actual,predicted)
#     rmse=np.sqrt(mean_squared_error(actual,predicted))
#     re_sqr=r2_score(actual,predicted)
#     # return mse,rmse,re_sqr
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
## Train multiple models
## Model Ecaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)

    mae, rmse,r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1016.0745189031425
MAE: 677.0746921777403
R2 score 93.72496005738562




Lasso
Model Training Performance
RMSE: 1016.335685324703
MAE: 678.2593222079579
R2 score 93.72173383664189


Ridge
Model Training Performance
RMSE: 1016.0623036352453
MAE: 677.0998229808058
R2 score 93.72511093378648


Elasticnet
Model Training Performance
RMSE: 1545.3121530891385
MAE: 1069.6205831723826
R2 score 85.48564255403446


